In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites require using proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.
This is an excellent use case for an LLM, because it requires nuanced understanding.

In [16]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""
#print(link_system_prompt)
# An example of one-shot prompting

In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

- https://platform.openai.com/docs/guides/migrate-to-responses
- The new 'Responses' API could be tried as well, instead of the older 'Chat Completions' API

### Chat Completions API:
```
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-5",
  messages=[
    {
      "role": "user",
      "content": "Write a one-sentence bedtime story about a unicorn."
    }
  ]
)
print(completion.choices[0].message.content)
```

### Responses API equivalent:
```
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
  model="gpt-5",
  input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)
```

In [9]:
#Can try this:
#huggingface = Website("https://huggingface.co")
#huggingface.links

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

#print(get_all_details("https://huggingface.co")

In [11]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

# get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [14]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [15]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}]}


# Hugging Face Company Brochure

**Welcome to Hugging Face – The AI Community Building the Future!**

## Who We Are
Hugging Face is a leading platform that fosters collaboration among the machine learning community. With a focus on transparency, innovation, and community engagement, we empower users to create, discover, and share cutting-edge AI models, datasets, and applications. 

## Our Offerings
- **Models**: Access over 1 million models, including state-of-the-art AI solutions such as transformers and diffusers.
- **Datasets**: Browse more than 250,000 datasets designed to cater to various machine learning tasks.
- **Spaces**: An interactive environment where users can run applications and collaborate on shared projects.
- **Enterprise Solutions**: Our tailored offerings for businesses include advanced compute options and enterprise-grade tools.

## Customers
Join over 50,000 organizations, including industry leaders like Google, Microsoft, Amazon, and Meta. Our platform is the go-to place for researchers, innovators, and enterprises looking to leverage machine learning technologies.

## Careers & Company Culture
At Hugging Face, we prioritize a diverse, inclusive workplace where creativity thrives. Our team is made up of passionate individuals who are not just employees but contributors to the broader AI community. We look for talent that shares our vision of making AI accessible and useful for everyone. 

**Why Join Us?**
- Work alongside experts in AI and machine learning.
- Contribute to open-source projects that are shaping the future.
- Access competitive benefits and a supportive work environment that encourages continuous learning.

## Join the Community
Be part of the AI transformation! Sign up today to start your journey with Hugging Face.

**Connect With Us**
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface/)
- [Discord](https://huggingface.co/discord)

**Start Exploring**: Systems designed for everyone from beginners to experts – let’s innovate together!

---

**Hugging Face: Together, Building Tomorrow's AI Today!**

# With a small adjustment, we can change this so that the results stream back from OpenAI, with the familiar typewriter animation

In [17]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

---

## Overview

**Hugging Face** is leading the charge in building a collaborative AI community focused on innovating the future through machine learning (ML). With a robust platform that hosts over **1 million models**, enormous datasets, and countless applications, it serves as the home for the machine learning community.

### Mission

Our mission is to empower everyone with machine learning capabilities, fostering a collaborative environment where individuals and organizations can build, share, and improve upon AI models and applications.

---

## What We Offer

### Collaboration Platform

- **Models**: Access a diverse set of **1M+ models** created and shared by the community.
- **Datasets**: Browse through over **250k+ datasets** tailored for various ML tasks.
- **Spaces**: Discover engaging applications that utilize advanced machine learning methods.

### Enterprise Solutions

- Enterprise-grade solutions featuring high-level security, dedicated support, and access control.
- Competitive pricing starting at **$20/user/month** with the compute options beginning at **$0.60/hour**.

### Open Source Commitment

At Hugging Face, we believe in the power of open-source technology. Our offerings include:

- **Transformers**: State-of-the-art AI models for PyTorch.
- **Tokenizers**: Fast and efficient tokenization for both research and production environments.
- **Diffusers** and much more.

---

## Customer Base

More than **50,000 organizations** leverage the Hugging Face platform, including major players like:

- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **AI2**

Our community includes enterprise clients, non-profits, and individual practitioners contributing to revolutionary research and deployment of ML technology.

---

## Company Culture

At Hugging Face, we're not just building AI; we’re cultivating a thriving community of innovators and passionate contributors. We prioritize:

- **Inclusivity**: Welcoming all backgrounds and skill levels.
- **Collaboration**: Working together to push the boundaries of what's possible in AI.
- **Open-Source Ethics**: Creating tools and resources that anyone can access and use.

---

## Careers at Hugging Face

We are constantly looking for talented individuals who are committed to shaping the future of AI. Joining us means being part of a dynamic force in machine learning, where:

- Creativity is encouraged.
- Professional growth is supported.
- Diversity and inclusion are held as core values.

Explore available job opportunities and become part of our transformative journey in the AI landscape.

---

## Join Us!

Whether you're an AI enthusiast, investor looking for the next big thing, or a professional seeking your next career opportunity, Hugging Face welcomes you to explore and collaborate in a unique and impactful environment.

### [Visit Us](https://huggingface.co) to learn more!

--- 

**Hugging Face – The AI Community Building the Future.**